In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm 


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), padding=(1,1))
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(in_features=8*8*512, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=256)
        self.Dropout = nn.Dropout(0.25)
        self.fc3 = nn.Linear(in_features=256, out_features=100)

    def forward(self, x):
        x = F.relu(self.conv1(x)) 
        x = F.relu(self.conv2(x)) 
        x = self.pool(x) 
        x = self.Dropout(x)
        x = F.relu(self.conv3(x)) 
        x = F.relu(self.conv4(x)) 
        x = self.pool(x) 
        x = self.Dropout(x)
        x = x.view(-1, 8*8*512) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.Dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
num_epochs = 50
batch_size = 50
learning_rate = 3e-4
num_workers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),

])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

cifar_trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainset_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batch_size,shuffle=True, num_workers=num_workers)

cifar_testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
testset_loader = torch.utils.data.DataLoader(cifar_testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(trainset_loader)):
 
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()


In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(trainset_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(testset_loader, model)*100:.2f}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix,accuracy_score, f1_score
def test_label_predictions(model, device, test_loader):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            prediction = output.argmax(dim=1, keepdim=True)
            actuals.extend(target.view_as(prediction))
            predictions.extend(prediction)
    return [i.item() for i in actuals], [i.item() for i in predictions]

actuals, predictions = test_label_predictions(model, device, testset_loader)
print('Confusion matrix:')
print(confusion_matrix(actuals, predictions))
print('F1 score: %f' % f1_score(actuals, predictions, average='macro'))
print('Accuracy score: %f' % accuracy_score(actuals, predictions))
print('Precision score : %f' % precision_score(actuals,predictions,average='macro'))
print('Recall score : %f' % recall_score(actuals,predictions, average='macro'))

Confusion matrix:
[[83  0  0 ...  0  0  0]
 [ 0 82  0 ...  0  0  0]
 [ 0  0 44 ...  0  4  0]
 ...
 [ 0  0  0 ... 64  0  0]
 [ 0  1  4 ...  0 34  0]
 [ 0  0  0 ...  0  0 67]]
F1 score: 0.593415
Accuracy score: 0.593100
Precision score : 0.600344
Recall score : 0.593100